In [54]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

ERROR:root:Line magic function `%install_ext` not found.


The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 4.8 ms


In [55]:
from collections import Counter
import difflib
import uuid
import itertools
import pandas as pd
import numpy as np
import string
import funcy
import re
import os


from dev import APP_DATA_DIR, NAME_COLS, PERSON_ID, NIH_ID, FEMALE_FIRST_NAMES, PICKLE_DIR

from data_cleaning_functions import has_suffix, remove_suffix_from_last_name 

time: 5.56 ms


In [56]:
filename = 'fuzzy_all_apps_plus_NIH_info.csv'

time: 3.28 ms


In [57]:
apps = pd.read_csv(os.path.join(APP_DATA_DIR, filename))

time: 65 ms


In [58]:
# for two people whom I manually added names for, fill in first and middle and last names
mask = pd.isnull(apps.last_name)
apps.loc[mask, 'last_name'] = apps[mask].clean_last_name.apply(lambda x: x.title())
mask = pd.isnull(apps.middle_name)
apps.loc[mask, 'middle_name'] = apps[mask].clean_middle_name.apply(lambda x: np.nan if pd.isnull(x) else x.title())
mask = pd.isnull(apps.first_name)
apps.loc[mask, 'first_name'] = apps[mask].clean_first_name.apply(lambda x: np.nan if pd.isnull(x) else x.title())

time: 13.2 ms


In [59]:
# generate a file with first initial of first name, and last name
# also generate a row with middle initial, and last name
# check for last names with spaces in them, mc last names, last names with an apostrophe
apps['first_initial'] = apps.clean_first_name.apply(lambda x: np.nan if pd.isnull(x) else str(x[0]))
apps['middle_initial'] = apps.clean_middle_name.apply(lambda x: np.nan if pd.isnull(x) else str(x[0]))
apps['clean_last_name2'] = apps.last_name.apply(lambda x: np.nan if pd.isnull(x) else x.upper().strip())
apps['clean_last_name3'] = np.nan

time: 17.4 ms


In [60]:
def find_space_in_name(raw_str):
    # finds strings with spaces in them
    if pd.isnull(raw_str):
        return False
    str_list = raw_str.split(' ')
    return len(str_list) >= 2

time: 6.15 ms


In [61]:
# for last names with a space in them, create version with and without space
odd_last_name = apps.clean_last_name.apply(find_space_in_name)

apps.loc[odd_last_name, 'clean_last_name3'] = apps[
    odd_last_name]['clean_last_name'].apply(lambda x: ''.join(x.split(' ')).strip())

apps.loc[apps.clean_last_name==apps.clean_last_name2, 'clean_last_name2'] = np.nan

mask = (pd.isnull(apps.clean_last_name2)) & ~(pd.isnull(apps.clean_last_name3))
apps.loc[mask, 'clean_last_name2'] = apps[mask]['clean_last_name3']

apps.loc[mask, 'clean_last_name3'] = np.nan

time: 23.4 ms


In [62]:
suff_mask = apps.clean_last_name2.apply(has_suffix)

time: 9.71 ms


In [63]:
# remove suffixes from their last name
# also remove any commas from the last name
apps.loc[suff_mask, 'clean_last_name2'] = apps[suff_mask]['clean_last_name2'].apply(remove_suffix_from_last_name)

time: 6.99 ms


In [64]:
def strip_commas(raw_str):
    if pd.isnull(raw_str):
        return np.nan
    punc = ('.', ',')
    return ''.join([x for x in raw_str if x not in punc])

time: 3.99 ms


In [65]:
apps['clean_last_name2'] = apps['clean_last_name2'].apply(strip_commas)

time: 11.7 ms


In [66]:
def make_hash_str(row):
    strs = row[['first_name', 'last_name', 'middle_name', 'address', 'original_medical_school']].values
    
    str_val = ''.join(map(str, strs))
    return uuid.uuid5(uuid.NAMESPACE_DNS, str_val)

time: 4.54 ms


In [67]:
apps['hash_id'] = apps.apply(make_hash_str, axis=1)

time: 1.8 s


In [68]:
apps.loc[apps.duplicated('hash_id', keep=False), NAME_COLS]

,clean_first_name,clean_middle_name,clean_last_name


time: 30.5 ms


In [69]:
apps['first_name2'] = apps.first_name.apply(lambda x: np.nan if pd.isnull(x) else x.upper().strip())
apps['middle_name2'] = apps.middle_name.apply(lambda x: np.nan if pd.isnull(x) else x.upper().strip())


time: 28.8 ms


In [70]:
apps.loc[apps.duplicated('hash_id', keep=False), NAME_COLS+[
        'dno', 'address', 'original_medical_school', 'med_school', 'hash_str', 'first_name', 'middle_name', 'last_name']]

,clean_first_name,clean_middle_name,clean_last_name,dno,address,original_medical_school,med_school,hash_str,first_name,middle_name,last_name


time: 40.9 ms


In [71]:
n2 = apps.loc[~(pd.isnull(apps.clean_last_name2)), :]
n3 = apps.loc[~(pd.isnull(apps.clean_last_name3)), :]

time: 7.36 ms


In [72]:
# concat all the rows together
all_versions = pd.concat([apps, n2, n3], axis=0).sort_values(['hash_id']+NAME_COLS)

time: 115 ms


In [73]:
# to csv 
all_versions.to_csv(os.path.join(APP_DATA_DIR, 'multi_name_versions_AMA_match.csv'))

time: 1.52 s
